## setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/cose474_final/BioRAG

/content/drive/MyDrive/cose474_final/BioRAG


In [4]:
!pip install transformers

In [5]:
!pip install sentence-transformers

In [6]:
!pip install faiss-cpu

In [7]:
!pip install langchain

In [26]:
# !git lfs install

Updated git hooks.
Git LFS initialized.


In [27]:
# !git clone https://huggingface.co/dmis-lab/biobert-v1.1

Cloning into 'biobert-v1.1'...
remote: Enumerating objects: 29, done.
remote: Total 29 (delta 0), reused 0 (delta 0), pack-reused 29
Unpacking objects: 100% (29/29), 108.30 KiB | 248.00 KiB/s, done.
Filtering content: 100% (2/2), 826.39 MiB | 38.71 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/cose474_final/BioRAG/biobert-v1.1/.git/hooks/post-checkout': Permission denied


In [12]:
!pip install boto3

In [11]:
# !git clone https://github.com/jind11/MMM-MCQA.git

In [28]:
cd MMM-MCQA

/content/drive/MyDrive/cose474_final/BioRAG/MMM-MCQA


## retriever (faiss trained with textbooks)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [9]:
faiss_path = './guideline'
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',
                                    model_kwargs={'device': 'cpu'})

db = FAISS.load_local(faiss_path, embeddings)

In [10]:
topk = db.similarity_search_with_score("what should I do when I have a headache?", k=1)
i, score = topk[0]
i.page_content

'SECTION 3: MANAGEMENT  -Patients with emergent red flag symptoms require immediate  emergency room evaluation:  • Onset of severe headache that is sudden (seconds to a minute to a  peak onset of intensity).  • Headache with fever and neck stiffness.  • Papilledema with altered level of consciousness and/or focal  neurological signs.  -Urgent red flag symptoms require prompt evaluation:  • Signs of systemic illness in the patient with new-onset headache.  • New headache in patients over 50 y of age with symptoms of  temporal arteritis.  • Papilledema in an alert patient without focal neurological signs.  • Elderly patient with new headache and subacute cognitive change.  -Consider imaging or specialty consultation for patients with the  following:  • Atypical headaches and changes in headache pattern.  • Unexplained focal signs in the patient with a headache.  • Headache precipitated by exertion, postural change, cough, or  valsalva.  • New-onset cluster headache or another trigeminal 

## medqa preprocessing 1 (DO NOT EXECUTE AGAIN)

already done preprocessing 1 & saved to csv

In [14]:
import pandas as pd

trainset_df = pd.read_json(path_or_buf='./phrases_no_exclude_train.jsonl', lines=True)
testset_df = pd.read_json(path_or_buf='./phrases_no_exclude_test.jsonl', lines=True)

In [15]:
trainset_df

,question,answer,options,meta_info,answer_idx,metamap_phrases
0,A 23-year-old pregnant woman at 22 weeks gesta...,Nitrofurantoin,"{'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': '...",step2&3,D,"[23 year old pregnant woman, weeks presents, b..."
1,A 3-month-old baby died suddenly at night whil...,Placing the infant in a supine position on a f...,{'A': 'Placing the infant in a supine position...,step2&3,A,"[3 month old baby died, night, asleep, mother,..."
2,A mother brings her 3-week-old infant to the p...,Abnormal migration of ventral pancreatic bud,{'A': 'Abnormal migration of ventral pancreati...,step1,A,"[mother, week old infant, pediatrician's offic..."
3,A pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,"{'A': 'Thromboembolism', 'B': 'Pulmonary ische...",step1,A,"[pulmonary autopsy specimen, 58 year old woman..."
4,A 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,"{'A': 'Hemophilia A', 'B': 'Lupus anticoagulan...",step1,D,"[20 year old woman presents, menorrhagia, past..."
...,...,...,...,...,...,...
10173,A 60-year-old man presents to the emergency de...,Administer desmopressin,"{'A': 'Administer demeclocycline', 'B': 'Admin...",step2&3,B,"[60 year old man presents, emergency departmen..."
10174,A 45-year-old male with a 15-year history of d...,Urine microalbumin to creatinine ratio,"{'A': 'Cystatin C levels', 'B': 'Urine microal...",step2&3,B,"[year old male, history of diabetes mellitus p..."
10175,After receiving a positive newborn screening r...,Acetaminophen,"{'A': 'Opioids', 'B': 'Acetaminophen', 'C': 'C...",step1,B,"[receiving, positive newborn, 2 week old male ..."
10176,A 25-year-old man comes to the office because ...,Homogentisic acid oxidase,{'A': 'Branched-chain alpha-ketoacid dehydroge...,step1,C,"[year old man, office, of pain in, left should..."


In [16]:
# question + answer pairs
def create_new_col(row):
    options_dict = row['options']
    options = [value for key, value in options_dict.items()]
    return [f"{row['question']} {option}" for option in options]

# apply the function to each row
trainset_df['qa_pairs'] = trainset_df.apply(create_new_col, axis=1)
testset_df['qa_pairs'] = testset_df.apply(create_new_col, axis=1)

In [17]:
# context
def create_context(row):
  question = row['question']
  retrieval = db.similarity_search_with_score(question, k=1)
  i, score = retrieval[0]
  context = i.page_content
  return context

# apply the function to each row
trainset_df['context'] = trainset_df.apply(create_context, axis=1)
testset_df['context'] = testset_df.apply(create_context, axis=1)

KeyboardInterrupt: ignored

In [ ]:
trainset_df.to_csv('./trainset.csv')
testset_df.to_csv('./testset.csv')

## medqa preprocessing 2 (DO NOT EXECUTE AGAIN)
convert medqa dataset into DREAM format

In [31]:
!ls '..'

bert_config.json  cose474_final.ipynb	   gpt3.ipynb  phrases_no_exclude_test.jsonl   testset.csv
bioASQ-factoid	  cose474_final_old.ipynb  guideline   phrases_no_exclude_train.jsonl  trainset.csv
biobert-v1.1	  encoding_train.csv	   MMM-MCQA    README.md		       vocab.txt


In [32]:
import pandas as pd

train = pd.read_csv('../trainset.csv')
test = pd.read_csv('../testset.csv')

test

,Unnamed: 0,question,answer,options,meta_info,answer_idx,metamap_phrases,qa_pairs,context
0,0,A junior orthopaedic surgery resident is compl...,Tell the attending that he cannot fail to disc...,{'A': 'Disclose the error to the patient and p...,step1,B,"['junior orthopaedic surgery resident', 'compl...",['A junior orthopaedic surgery resident is com...,"patient has a nonurgent condition, the clinici..."
1,1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,"['67 year old man', 'transitional cell carcino...","[""A 67-year-old man with transitional cell car...",c . More difficulty with high-frequency sound...
2,2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,"['Two weeks', 'emergency cardiac', 'stenting',...",['Two weeks after undergoing an emergency card...,CASE 81 hX P e ddX 40 yo M with a history of d...
3,3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,"['year old woman', 'brought', 'emergency depar...",['A 39-year-old woman is brought to the emerge...,CASE 99 hX P e ddX 6 yo M is brought by his mo...
4,4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,"['35 year old man', 'physician', 'of itchy', '...",['A 35-year-old man comes to the physician bec...,b . Causes focal unilateral blotchy redness o...
...,...,...,...,...,...,...,...,...,...
1268,1268,A 39-year-old woman presents to the clinic for...,Thyroid-stimulating hormone (TSH),"{'A': 'Glucose', 'B': 'Triiodothyronine (T3)',...",step2&3,D,"['year old woman presents', 'clinic', 'annual ...",['A 39-year-old woman presents to the clinic f...,18 . A 38-year-old female presents to your of...
1269,1269,A 38-year-old woman comes to the physician bec...,Medication abuse,"{'A': 'Celiac disease', 'B': 'Carcinoid tumor'...",step2&3,D,"['year old woman', 'physician', 'persistent di...",['A 38-year-old woman comes to the physician b...,22 . A 32-year-old female presents to your of...
1270,1270,A 21-year-old college student comes to the phy...,Stop evening exercise,"{'A': 'Trial of diphenhydramine', 'B': 'Stop e...",step2&3,B,"['21-year-old college student', 'physician', '...",['A 21-year-old college student comes to the p...,Sleep-Wake Disorders INSOMNIA DISORDER Defined...
1271,1271,A 19-year-old man is brought to the physician ...,Schizotypal personality disorder,"{'A': 'Social anxiety disorder', 'B': 'Avoidan...",step2&3,C,"['year old man', 'brought', 'physician', 'moth...",['A 19-year-old man is brought to the physicia...,Psychiatry ` Psychiatry—Pathology Psychiatry ...


In [33]:
bottom = train.index[-1273:]
dev = train.loc[bottom]
train = train.drop(bottom)

# train : 8905, dev : 1273, test : 1273

In [35]:
with open('../MMM-MCQA/data/dream/train.json') as rf:
  dream_train = rf.read()
with open('../MMM-MCQA/data/dream/dev.json') as rf:
  dream_dev = rf.read()
with open('../MMM-MCQA/data/dream/test.json') as rf:
  dream_test = rf.read()
dream_train

Output hidden; open in https://colab.research.google.com to view.

In [36]:
eval(dream_train)[0]

[['man :  I am considering dropping my dancing class. I am not making any progress.',
  "woman :  If I were you, I stick with it. It's definitely worth time and effort."],
 [{'question': 'What does the man suggest the woman do?',
   'choice': ['Consult her dancing teacher.',
    'Take a more interesting class.',
    'Continue her dancing class.'],
   'answer': 'Continue her dancing class.'}],
 '5-510']

In [56]:
import json

def process(dataset):
  total = []
  subtotal = []
  qca_dict = dict()

  for index, row in dataset.iterrows():
    question = row['question']
    context = row['context']
    choice_dict = eval(row['options'])
    answer = row['answer']
    choices = list(choice_dict.values())

    qca_dict = {
        'question' : question,
        'choice' : choices,
        'answer' : answer
    }

    subtotal = [[context], [qca_dict], question]
    total.append(subtotal)

  return total

In [57]:
train_new = process(train)
dev_new = process(dev)
test_new = process(test)

In [63]:
import json

with open('./data/dream/train.json', 'w') as wf:
  json.dump(train_new, wf)
with open('./data/dream/dev.json', 'w') as wf:
  json.dump(dev_new, wf)
with open('./data/dream/test.json', 'w') as wf:
  json.dump(test_new, wf)

In [66]:
with open('./data/dream/test.json') as rf:
  data = json.load(rf)

data[0]

[['patient has a nonurgent condition, the clinician should not recommend that the patient visit the emergency department.  MNEMONIC The 4 Ds of malpractice— D uty D ereliction D amage D irect cause IMPAIRED PRACTICING CLINICIAN'],
 [{'question': 'A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?',
   'choice': ['Disclose the error to the patient and put it in the operative report',
    'Tell the attending that he cannot fail to disclose 

In [68]:
with open('./data/dream_old/test.json') as rf:
  tmp = json.load(rf)

tmp[0]

[['woman :  The movie next Tuesday has been cancelled due to lack of interest.',
  'man :  What do you mean?',
  'woman :  Well, by last night only a few tickets has been sold.'],
 [{'question': 'What can we conclude about the movie?',
   'choice': ['They want to buy the tickets for the movie.',
    'The tickets for the movie were sold.',
    'The movie will not be shown.'],
   'answer': 'The movie will not be shown.'}],
 '4-199']

## train

In [69]:
!python run_classifier_bert_exe.py dream ../biobert-v1.1 1 4

12/05/2023 15:52:44 - INFO - __main__ -   device cpu n_gpu 0 distributed training False
12/05/2023 15:52:44 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file ../biobert-v1.1/vocab.txt
12/05/2023 15:52:44 - INFO - pytorch_pretrained_bert.modeling -   loading weights file ../biobert-v1.1/pytorch_model.bin
12/05/2023 15:52:44 - INFO - pytorch_pretrained_bert.modeling -   loading configuration file ../biobert-v1.1/config.json
12/05/2023 15:52:44 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

12/05/20